In [11]:
import sys
#sys.path.append("../")
from src import prepro, metrics, run, setup
import src.models.factory as model_factory
import config
import torch

import numpy as np
from src.utils import models
import os.path as osp
import pandas as pd

from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from src.utils.data_processing import *

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#args = setup.create_setup()
#args.mode = 'displacement'
#args.full_encoder = True
#args.encoder_config = 'full_encoder_config'
#args.decoder_config = 'lstm_config'
window_size = 8
predict_at = 16
#args.target_intensity_cat = False
#args.sub_window_size = 8
#args.sub_area = 1
#args.output_dir = '../results/results8_16_20_44_28' #''../results/results8_16_18_57_7' best so far
#try: ../results/results8_16_20_44_28, 0.086 from training
#Reached 70.4 and the best results with combination. 7.47 GRU outputs 64. All rest normal.
#args.output_dir = './results/results7_20_17_12_19' #Be careful to change 2304 for intermediary layer
#args.output_dir = './results/results7_20_15_4_36' : best for acc 70.5 and nearly best for intensiy 7.50
#args. = ./results/results8_12_19_5_42 for best perf t+48h

#if args.sub_area > 0:
    #x_viz_train = x_viz_train[:, :, :, args.sub_area:-args.sub_area, args.sub_area:-args.sub_area]
    #x_viz_test = x_viz_test[:, :, :, args.sub_area:-args.sub_area, args.sub_area:-args.sub_area]

tgt_intensity_cat_train = torch.LongTensor(np.load('../data/y_train_intensity_cat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                      allow_pickle=True))
tgt_intensity_cat_test = torch.LongTensor(np.load('../data/y_test_intensity_cat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))

tgt_intensity_train = torch.Tensor(np.load('../data/y_train_intensity_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                  allow_pickle=True))
tgt_intensity_test = torch.Tensor(np.load('../data/y_test_intensity_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                 allow_pickle=True))

tgt_intensity_cat_baseline_train = torch.LongTensor(np.load('../data/y_train_intensity_cat_baseline_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',  allow_pickle = True))
tgt_intensity_cat_baseline_test = torch.LongTensor(np.load('../data/y_test_intensity_cat_baseline_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))

tgt_displacement_train = torch.Tensor(np.load('../data/y_train_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))
tgt_displacement_test = torch.Tensor(np.load('../data/y_test_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                    allow_pickle=True))

tgt_displacement_train_unst = torch.Tensor(np.load('../data/y_train_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))
tgt_displacement_test_unst = torch.Tensor(np.load('../data/y_test_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                    allow_pickle=True))


mean_intensity = tgt_intensity_train.mean()
std_intensity = tgt_intensity_train.std()
tgt_intensity_train = (tgt_intensity_train - mean_intensity)/std_intensity
tgt_intensity_test = (tgt_intensity_test - mean_intensity)/std_intensity


###INTENSITY
mean_dx = tgt_displacement_train[:,0].mean()
std_dx = tgt_displacement_train[:,0].std()
tgt_displacement_train[:,0] = (tgt_displacement_train[:,0] - mean_dx)/std_dx
tgt_displacement_test[:,0] = (tgt_displacement_test[:,0] - mean_dx)/std_dx
std_dx = float(std_dx)
mean_dx = float(mean_dx)

mean_dy = tgt_displacement_train[:,1].mean()
std_dy = tgt_displacement_train[:,1].std()
tgt_displacement_train[:,1] = (tgt_displacement_train[:,1] - mean_dy)/std_dy
tgt_displacement_test[:,1] = (tgt_displacement_test[:,1] - mean_dy)/std_dy
std_dy = float(std_dy)
mean_dy = float(mean_dy)

def standardize(tgt_displacement_train, tgt_displacement_test):
    mean_dx = tgt_displacement_train[:, 0].mean()
    std_dx = tgt_displacement_train[:, 0].std()
    tgt_displacement_train[:, 0] = (tgt_displacement_train[:, 0] - mean_dx) / std_dx
    tgt_displacement_test[:, 0] = (tgt_displacement_test[:, 0] - mean_dx) / std_dx
    std_dx = float(std_dx)
    mean_dx = float(mean_dx)
    mean_dy = tgt_displacement_train[:, 1].mean()
    std_dy = tgt_displacement_train[:, 1].std()
    tgt_displacement_train[:, 1] = (tgt_displacement_train[:, 1] - mean_dy) / std_dy
    tgt_displacement_test[:, 1] = (tgt_displacement_test[:, 1] - mean_dy) / std_dy
    std_dy = float(std_dy)
    mean_dy = float(mean_dy)
    return tgt_displacement_train, tgt_displacement_test, std_dx, mean_dx, std_dy, mean_dy

def unstandardize(tgt_displacement_train, tgt_displacement_test, std_dx, mean_dx, std_dy, mean_dy):
    tgt_displacement_train[:, 0] = tgt_displacement_train[:, 0] *  std_dx + mean_dx
    tgt_displacement_test[:, 0] = tgt_displacement_test[:, 0] *  std_dx + mean_dx
    tgt_displacement_train[:, 1] = tgt_displacement_train[:, 1] * std_dy + mean_dy
    tgt_displacement_test[:, 1] = tgt_displacement_test[:, 1] * std_dy + mean_dy
    return tgt_displacement_train, tgt_displacement_test


##########
##########
########## PREPARING DATA FOR XGB

X_train = np.load('../data/X_train_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)
X_test = np.load('../data/X_test_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)


names = ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND',
         'STORM_SPEED', 'cat_cos_day', 'cat_sign_day', 'COS_STORM_DIR', 'SIN_STORM_DIR',
         'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'cat_storm_category', 'cat_basin_AN',
         'cat_basin_EP', 'cat_basin_NI', 'cat_basin_SA',
         'cat_basin_SI', 'cat_basin_SP', 'cat_basin_WP', 'cat_nature_DS', 'cat_nature_ET',
         'cat_nature_MX', 'cat_nature_NR', 'cat_nature_SS', 'cat_nature_TS',
         'STORM_DISPLACEMENT_X', 'STORM_DISPLACEMENT_Y']

names_all = names * window_size

for i in range(len(names_all)):
    names_all[i] += '_' + str(i // 30)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.columns = names_all
X_test.columns = names_all

cols = [c for c in X_train.columns if c.lower()[-2:] == '_0' or c.lower()[:3] != 'cat']

X_train = X_train[cols]
X_test = X_test[cols]

In [60]:
#X_train_embed = np.load('../data/embeddings/X_train_embeds_1980_34_20_120_results8_16_20_44_28.npy', allow_pickle = True)
#X_test_embed = np.load('../data/embeddings/X_test_embeds_1980_34_20_120_results8_16_20_44_28.npy', allow_pickle = True)
#X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_intensity.npy', allow_pickle = True)
#X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_intensity.npy', allow_pickle = True)

#48
X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_intensity_48.npy', allow_pickle = True)
X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_intensity_48.npy', allow_pickle = True)

#X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_track_48.npy', allow_pickle = True)
#X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_track_48.npy', allow_pickle = True)

X_train_total = np.concatenate((X_train, X_train_embed), axis = 1)
X_test_total = np.concatenate((X_test, X_test_embed), axis = 1)

std_ = float(std_intensity)
mean_ = float(mean_intensity)

In [56]:
X_train_embed.std(axis=(0))

array([0.32623637, 0.2453524 , 0.30252   , 0.2068524 , 0.569222  ,
       0.3129427 , 0.25083327, 0.5475748 , 0.45903492, 0.29479325,
       0.39042005, 0.46348616, 0.39096308, 0.33948642, 0.24545877,
       0.28124598, 0.28033993, 0.42566696, 0.51634353, 0.5100166 ,
       0.30813473, 0.476275  , 0.3991406 , 0.3804275 , 0.4139351 ,
       0.29657942, 0.5055465 , 0.3844849 , 0.15621336, 0.44216263,
       0.19743301, 0.45874766, 0.5273378 , 0.32123196, 0.38916793,
       0.5022551 , 0.2860133 , 0.27160797, 0.28035945, 0.5265858 ,
       0.4426414 , 0.3194403 , 0.49691105, 0.29644164, 0.5246908 ,
       0.42100573, 0.42218858, 0.44222334, 0.37747246, 0.55683464,
       0.4348592 , 0.5397909 , 0.26551577, 0.25035217, 0.25046262,
       0.44206345, 0.3987248 , 0.44384882, 0.4536024 , 0.36178795,
       0.34023088, 0.5061376 , 0.3568393 , 0.37379837, 0.47772896,
       0.31256968, 0.3720116 , 0.26771006, 0.55373466, 0.35055223,
       0.3148587 , 0.53132206, 0.5260325 , 0.3746377 , 0.52086

In [ ]:
xgb2 = XGBRegressor(max_depth=6, n_estimators=140, learning_rate = 0.07, subsample = 0.7, min_child_weight = 5)
xgb2.fit(X_train, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb2.predict(X_test))*std_+mean_))

xgb = XGBRegressor(max_depth=8, n_estimators = 150, learning_rate = 0.07, subsample = 0.9)
xgb.fit(X_train_total, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_total))*std_+mean_))

In [62]:
xgb = XGBRegressor(max_depth=8, n_estimators = 150, learning_rate = 0.07, subsample = 0.7)
xgb.fit(X_train_embed, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_embed))*std_+mean_))

MAE intensity:  22.123682


In [65]:
xgb = XGBRegressor(max_depth=8, n_estimators = 120, learning_rate = 0.07, subsample = 0.7)
xgb.fit(X_train_total, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_total))*std_+mean_))

MAE intensity:  20.64371


In [64]:
xgb.predict(X_train_total)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [66]:
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_train)*std_+mean_, np.array(xgb.predict(X_train_total))*std_+mean_))

MAE intensity:  6.808701


In [51]:
X_train_embed.std(axis=(0))

array([0.32623637, 0.2453524 , 0.30252   , 0.2068524 , 0.569222  ,
       0.3129427 , 0.25083327, 0.5475748 , 0.45903492, 0.29479325,
       0.39042005, 0.46348616, 0.39096308, 0.33948642, 0.24545877,
       0.28124598, 0.28033993, 0.42566696, 0.51634353, 0.5100166 ,
       0.30813473, 0.476275  , 0.3991406 , 0.3804275 , 0.4139351 ,
       0.29657942, 0.5055465 , 0.3844849 , 0.15621336, 0.44216263,
       0.19743301, 0.45874766, 0.5273378 , 0.32123196, 0.38916793,
       0.5022551 , 0.2860133 , 0.27160797, 0.28035945, 0.5265858 ,
       0.4426414 , 0.3194403 , 0.49691105, 0.29644164, 0.5246908 ,
       0.42100573, 0.42218858, 0.44222334, 0.37747246, 0.55683464,
       0.4348592 , 0.5397909 , 0.26551577, 0.25035217, 0.25046262,
       0.44206345, 0.3987248 , 0.44384882, 0.4536024 , 0.36178795,
       0.34023088, 0.5061376 , 0.3568393 , 0.37379837, 0.47772896,
       0.31256968, 0.3720116 , 0.26771006, 0.55373466, 0.35055223,
       0.3148587 , 0.53132206, 0.5260325 , 0.3746377 , 0.52086

In [18]:
xgb = XGBRegressor(max_depth=8, n_estimators = 140, learning_rate = 0.07, subsample = 0.8)
xgb.fit(X_train_total, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_total))*std_+mean_))

MAE intensity:  11.97465


In [22]:
!ls ../data/

embeddings
last3years.csv
tensor_completion
vision_data_1980_34_20_120_forecast_all_48_clean.npy
vision_data_1980_34_20_120.npy
X_test_stat_1980_34_20_120_forecast_24_w8_at_8.npy
X_test_stat_1980_34_20_120_forecast_all_48_clean_w8_at_16.npy
X_test_stat_1980_34_20_120_w16_at_8.npy
X_test_stat_1980_34_20_120_w4_at_16.npy
X_test_stat_1980_34_20_120_w4_at_8.npy
X_test_stat_1980_34_20_120_w8_at_16.npy
X_test_stat_1980_34_20_120_w8_at_8.npy
X_test_stat_1980_34_20_120_withforecast_2661_w8_at_8.npy
X_test_stat_with_dates_columns_1980_w8_at_8.npy
X_test_vision_1980_34_20_120_w12_at_8.npy
X_test_vision_1980_34_20_120_w16at_16.npy
X_test_vision_1980_34_20_120_w16_at_24.npy
X_test_vision_1980_34_20_120_w16_at_8.npy
X_test_vision_1980_34_20_120_w2_at_8.npy
X_test_vision_1980_34_20_120_w4_at_16.npy
X_test_vision_1980_34_20_120_w4_at_8.npy
X_test_vision_1980_34_20_120_w8_at_16.npy
X_test_vision_1980_34_20_120_w8_at_8.npy
X_test_vision_1980_50_20_90_w12.npy
X_test_vision_1980_50_20_90_w16.npy
X_test_v

In [40]:
#X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_24_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))
X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_all_48_clean_w8_at_16.npy', allow_pickle=True))

#X_test_dates = pd.DataFrame(np.load('../data/X_test_stat_with_dates_columns_1980_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True)[:,:4])

#X_test_dates.columns = ['YEAR', 'MONTH', 'DAY', 'HOUR']

names_baselines = ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'cos_day', 'sin_day', 'COS_STORM_DIR', 'SIN_STORM_DIR', 'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'wind_category', 'OFCL_24_lat', 'OFCL_24_lon', 'OFCL_24_vmax', 'OFCL_24_mslp', 'P91E_24_lat', 'P91E_24_lon', 'P91E_24_vmax', 'P91E_24_mslp', 'GFDL_24_lat', 'GFDL_24_lon', 'GFDL_24_vmax', 'GFDL_24_mslp', 'NGPS_24_lat', 'NGPS_24_lon', 'NGPS_24_vmax', 'NGPS_24_mslp', 'LBAR_24_lat', 'LBAR_24_lon', 'LBAR_24_vmax', 'LBAR_24_mslp', 'GUNS_24_lat', 'GUNS_24_lon', 'GUNS_24_vmax', 'GUNS_24_mslp', 'UKXI_24_lat', 'UKXI_24_lon', 'UKXI_24_vmax', 'UKXI_24_mslp', 'EMXI_24_lat', 'EMXI_24_lon', 'EMXI_24_vmax', 'EMXI_24_mslp', 'A98E_24_lat', 'A98E_24_lon', 'A98E_24_vmax', 'A98E_24_mslp', 'SHIP_24_lat', 'SHIP_24_lon', 'SHIP_24_vmax', 'SHIP_24_mslp', 'DSHP_24_lat', 'DSHP_24_lon', 'DSHP_24_vmax', 'DSHP_24_mslp', 'FSSE_24_lat', 'FSSE_24_lon', 'FSSE_24_vmax', 'FSSE_24_mslp', 'CLP5_24_lat', 'CLP5_24_lon', 'CLP5_24_vmax', 'CLP5_24_mslp', 'AEMN_24_lat', 'AEMN_24_lon', 'AEMN_24_vmax', 'AEMN_24_mslp', 'CMC_24_lat', 'CMC_24_lon', 'CMC_24_vmax', 'CMC_24_mslp', 'GFSO_24_lat', 'GFSO_24_lon', 'GFSO_24_vmax', 'GFSO_24_mslp', 'HWRF_24_lat', 'HWRF_24_lon', 'HWRF_24_vmax', 'HWRF_24_mslp', 'cat_basin_AN', 'cat_basin_EP', 'basin_NI', 'basin_SA', 'basin_SI', 'cat_basin_SP', 'cat_basin_WP', 'DISPLACEMENT_LAT', 'DISPLACEMENT_LON']

names_all_baselines = names_baselines * 8#args.window_size

for i in range(len(names_all_baselines)):
    names_all_baselines[i] += '_' + str(i // 92)

X_test_baseline.columns = names_all_baselines

#X_test_baseline = pd.concat([X_test_baseline, X_test_dates], axis = 1)

In [32]:
EP:
We beat: SHIP, HWRF, GFSO, AEMN, DSHP, (GFDL, UKXI, CMC)
We lose: OFCL, FSSE

AN:
We beat: SHIP, HWRF, GFSO, AEMN, GFDL, DSHP, (UKXI, CMC) 
We lose: HWRF, OFCL, FSSE

(26800, 740)

In [32]:
train = X_train_total
    #test = X_test_total
tgt_train = tgt_intensity_train

xgb_total = XGBRegressor(max_depth=8, n_estimators=150, learning_rate=0.07, subsample=0.8, min_child_weight=1)
xgb_total.fit(train, tgt_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.07, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=150, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=0.8,
       tree_method='exact', validate_parameters=1, verbosity=None)

In [45]:
compare_perf_intensity(xgb_total = xgb_total, basin='EP', forecast='FSSE', mode='vmax', forecast2 = 'OFCL')

Timesteps 1368
MAE intensity basin EP Hurricast :  15.04 with std  19.6
MAE intensity basin EP Official Forecast FSSE :  15.78 with std  18.86
MAE intensity basin EP Official Forecast OFCL :  17.37 with std  20.59
Percentage of missed intensification > 20kn Hurricast:  28.44
Percentage of missed intensification > 20kn Official ForecastFSSE :  29.31
Percentage of missed intensification > 20kn Official Forecast 2OFCL :  30.41

MAE intensity basin EP Hurricast last 1000 :  15.54
MAE intensity basin EP Official Forecast FSSE :  15.6


In [46]:
train_xgb_intensity(forecast = 'EMXI', basin_only = False, sparse = False, max_depth = 8, n_estimators = 150, learning_rate = 0.07, subsample = 0.8, min_child_weight=1, basin = 'EP', forecast2 = 'OFCL')

Timesteps 0


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [28]:
train_x = X_train_total
train_y = X_train_total
test_x = X_test_total
test_y = X_test_total
tgt_train = tgt_displacement_train

xgb_x = XGBRegressor(max_depth=7, n_estimators=90, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_x.fit(train_x, tgt_train[:, 0])
xgb_y = XGBRegressor(max_depth=7, n_estimators=90, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_y.fit(train_y, tgt_train[:, 1])
DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
LATS_PRED_ = X_test['LAT_7'] + DLATS_PRED
LONS_PRED_ = X_test['LON_7'] + DLONS_PRED

In [ ]:
EP: Intensity
We beat: HWRF, SHIP, GFSO, AEMN, DSHP, (GFDL, UKXI, CMC)
We lose: OFCL, FSSE

In [ ]:
EP: Track
We beat: GFDL, CMC, CLP5
We lose: FSSE, OFCL, SHIP, HWRF, GFSO, AEMN, DSHP (UKXI)
    
AN: Track
We beat: GFDL, CMC, CLP5
We lose: FSSE, OFCL, SHIP, HWRF, GFSO, AEMN, DSHP (UKXI)

In [ ]:
compare_perf_track(basin='EP', forecast='SHIP', forecast2 = 'HWRF', LATS_PRED_=LATS_PRED_, LONS_PRED_=LONS_PRED_)

In [26]:
def train_xgb_track(basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    train_x = X_train_total
    train_y = X_train_total
    test_x = X_test_total
    test_y = X_test_total
    tgt_train = tgt_displacement_train
    if sparse:
        train_x, train_y = X_train_total_sparse_x, X_train_total_sparse_y
        test_x, test_y = X_test_total_sparse_x, X_test_total_sparse_y
    if basin_only:
        train_x = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        train_y = train_x
        tgt_train = tgt_displacement_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_x.fit(train_x, tgt_train[:, 0])
    xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_y.fit(train_y, tgt_train[:, 1])
    DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
    DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
    LATS_PRED_ = X_test['LAT_7'] + DLATS_PRED
    LONS_PRED_ = X_test['LON_7'] + DLONS_PRED
    compare_perf_track(basin=basin, forecast=forecast, forecast2 = forecast2, LATS_PRED_=LATS_PRED_, LONS_PRED_=LONS_PRED_)

def train_xgb_track_all_years(use_forecast = False, basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    train_x = X_train_total
    train_y = X_train_total
    test_x = X_test_total
    test_y = X_test_total
    tgt_train = tgt_displacement_train
    if sparse:
        train_x, train_y = X_train_total_sparse_x, X_train_total_sparse_y
        test_x, test_y = X_test_total_sparse_x, X_test_total_sparse_y
    if basin_only:
        train_x = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        train_y = train_x
        tgt_train = tgt_displacement_train[X_train['cat_basin_'+basin+'_0'] == 1]
    if use_forecast:
        train_for = X_train_forecasts
        tgt_train_for = tgt_train_dis_forecasts
        test_for = X_test_forecasts
        xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
        xgb_x.fit(train_for, tgt_train_for[:, 0])
        xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
        xgb_y.fit(train_for, tgt_train_for[:, 1])
        DLATS_PRED = np.array(xgb_x.predict(X_new)) * std_dx + mean_dx
        DLONS_PRED = np.array(xgb_y.predict(X_new)) * std_dy + mean_dy
    else:
        xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
        xgb_x.fit(train_x, tgt_train[:, 0])
        xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
        xgb_y.fit(train_y, tgt_train[:, 1])
        DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
        DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
    LATS_PRED_2012 = X_test['LAT_7'] + DLATS_PRED
    LONS_PRED_2012 = X_test['LON_7'] + DLONS_PRED
    compare_perf_track(basin=basin, forecast=forecast, forecast2 = forecast2, LATS_PRED_=LATS_PRED_2012, LONS_PRED_=LONS_PRED_2012)
    dict = {'year': [], 'num_samples': [], 'MAEs_full': [], 'std_full': [], 'MAES_2012': [], 'std_2012': [],
            'MAES_SHIP': [], 'std_SHIP': [], 'MAES_HWRF': [], 'std_HWRF': []}
    for year in range(2012, 2020):
        try:
            index = X_test_baseline.loc[
                X_test_baseline['YEAR'] < year].index
            X_test_to_train = X_test_total[index]
            train = np.concatenate((X_train_total, X_test_to_train), axis=0)
            tgt_train = np.concatenate((tgt_displacement_train, tgt_displacement_test[index]), axis=0)
            xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                 subsample=subsample, min_child_weight=min_child_weight)
            xgb_x.fit(train, tgt_train[:, 0])
            xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                 subsample=subsample, min_child_weight=min_child_weight)
            xgb_y.fit(train, tgt_train[:, 1])
            DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
            DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
            LATS_PRED_ = X_test['LAT_7'] + DLATS_PRED
            LONS_PRED_ = X_test['LON_7'] + DLONS_PRED
            compare_perf_track_per_year(dict, LATS_PRED_, LONS_PRED_, LATS_PRED_2012, LONS_PRED_2012, forecast=forecast,
                                            forecast2=forecast2, basin=basin,  year=year)
            print("\n")
        except:
            print("\n No forecasts for year ", year)
    return dict




def train_xgb_intensity(forecast = 'SHIP', basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast2 = None):
    train = X_train_total
    #test = X_test_total
    tgt_train = tgt_intensity_train
    if sparse:
        train = X_train_total_sparse_x
        test = X_test_total_sparse_x
    if basin_only:
        train = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        tgt_train = tgt_intensity_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_total.fit(train, tgt_train)
    compare_perf_intensity(xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', forecast2 = forecast2)

#train_xgb_track(n_estimators = 90, max_depth = 7, learning_rate = 0.12, subsample = 0.7, min_child_weight = 5, basin = 'AN', forecast = 'HWRF') 82.14 and 117.26
#train_xgb_track(n_estimators = 90, max_depth = 7, learning_rate = 0.1, subsample = 0.7, min_child_weight = 5, basin = 'EP', forecast = 'HWRF')

LATS_TEST = X_test['LAT_7'] + np.array(tgt_displacement_test[:,0])*std_dx+mean_dx
LONS_TEST = X_test['LON_7'] + np.array(tgt_displacement_test[:,1])*std_dy+mean_dy

def compare_perf_track(LATS_PRED_, LONS_PRED_, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    mode = 'lat'
    if forecast2 != None:
        index = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        LATS_BASE_2 = np.array(baseline_[forecast2 + '_24_lat_7'])
        LONS_BASE_2 = np.array(baseline_[forecast2 + '_24_lon_7'])
    else:
        index = X_test_baseline.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1].index#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    LATS_TEST_ = X_test['LAT_7'] + np.array(tgt_displacement_test[:, 0])*std_dx+mean_dx
    LONS_TEST_ = X_test['LON_7'] + np.array(tgt_displacement_test[:, 1])*std_dy+mean_dy
    baseline_1_x = baseline_[forecast + '_24_'+mode+'_7']
    baseline_1_y = baseline_[forecast + '_24_lon_7']
    LATS_BASE = np.array(baseline_1_x)
    LONS_BASE = np.array(baseline_1_y)
    LATS_TEST_ = np.array(LATS_TEST_[index])
    LONS_TEST_ = np.array(LONS_TEST_[index])
    LATS_PRED_ = np.array(LATS_PRED_[index])
    LONS_PRED_ = np.array(LONS_PRED_[index])
    d_km_baseline = np.zeros(len(LATS_BASE))
    d_km_baseline2 = np.zeros(len(LATS_BASE))
    d_km_pred = np.zeros(len(LONS_PRED_))
    for i in range(len(LATS_BASE)):
        d_km_baseline[i] = get_distance_km(LONS_BASE[i], LATS_BASE[i], LONS_TEST_[i], LATS_TEST_[i])
        d_km_pred[i] = get_distance_km(LONS_PRED_[i], LATS_PRED_[i], LONS_TEST_[i], LATS_TEST_[i])
        if forecast2 != None:
            d_km_baseline2[i] = get_distance_km(LONS_BASE_2[i], LATS_BASE_2[i], LONS_TEST_[i], LATS_TEST_[i])
    print("Number of timesteps:", len(LATS_BASE))
    print(basin, 'Model | MAE | std')
    print(forecast, np.around(d_km_baseline.mean(), decimals = 2), np.around(d_km_baseline.std(), decimals = 2))
    print(str(forecast2), np.around(d_km_baseline2.mean(), decimals = 2), np.around(d_km_baseline2.std(), decimals = 2))
    print("Hurricast", np.around(d_km_pred.mean(), decimals = 2), np.around(d_km_pred.std(), decimals = 2))
    print("\nModel | Number of Busts > 200km | Percentage Bust")
    print(forecast, sum(d_km_baseline > 200), np.around(sum(d_km_baseline > 200)*100/len(LATS_BASE), decimals = 2))
    print(str(forecast2), sum(d_km_baseline2 > 200), np.around(sum(d_km_baseline2 > 200)*100/len(LATS_BASE), decimals =2))
    print("Hurricast", sum(d_km_pred > 200), np.around(sum(d_km_pred > 200)*100/len(LATS_BASE), decimals = 2))



def compare_perf_intensity(xgb_total, basin = 'AN', forecast = 'SHIP', last_storms = 1000, mode = 'vmax', forecast2 = 'HWRF'):
    if forecast2 != None:
        index = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        # X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_2 = baseline_[forecast2 + '_24_' + mode + '_7']
    else:
        index = X_test_baseline.loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        # X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    X_test_withBASELINE_total = X_test_total[index]
    baseline_1 = baseline_[forecast + '_24_' + mode + '_7']
    if mode == 'vmax':
        tgt_ = np.array(tgt_intensity_test[index] * std_ + mean_)
        preds = xgb_total.predict(X_test_withBASELINE_total) * std_ + mean_
        #print("MAE intensity basin " + basin + " X stat vs "+ forecast + " : ", mean_absolute_error(tgt_intensity_test_withBASELINE * std_ + mean_,
                                                     #xgb.predict(X_test_withBASELINE) * std_ + mean_))
        print("Timesteps", len(tgt_))
        print("MAE intensity basin " + basin + " Hurricast : ", np.around(mean_absolute_error(tgt_, preds), decimals = 2), "with std ", np.around(np.std(tgt_ - preds), decimals=2))
        print("MAE intensity basin " + basin + " Official Forecast "+ forecast + " : ",
              np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2), "with std ", np.around(np.std(tgt_ - baseline_1), decimals = 2))
        print("MAE intensity basin " + basin + " Official Forecast " + str(forecast2) + " : ",
              np.around(mean_absolute_error(tgt_, baseline_2), decimals=2), "with std ",
              np.around(np.std(tgt_ - baseline_2), decimals=2))
        print("Percentage of missed intensification > 20kn Hurricast: ", np.around(sum(abs(tgt_ - preds) > 20)/len(preds) * 100, decimals = 2))
        print("Percentage of missed intensification > 20kn Official Forecast"+ forecast + " : ", np.around(sum(abs(tgt_ - baseline_1) > 20) / len(baseline_1) * 100, decimals =2))
        print("Percentage of missed intensification > 20kn Official Forecast 2"+ str(forecast2) + " : ", np.around(sum(abs(tgt_ - baseline_2) > 20) / len(baseline_2) * 100, decimals =2))
        print("\nMAE intensity basin " + basin + " Hurricast last", last_storms, ": ", np.around(mean_absolute_error(tgt_[-last_storms:], preds[-last_storms:]), decimals=2))
        print("MAE intensity basin " + basin + " Official Forecast " + forecast + " : ",
              np.around(mean_absolute_error(tgt_[-last_storms:], baseline_1[-last_storms:]), decimals=2))


#train_xgb_intensity(forecast = 'SHIP', basin = 'EP', max_depth=8, n_estimators = 120, learning_rate = 0.07, subsample = 0.8, min_child_weight = 1)
#train_xgb_intensity(forecast = 'SHIP', basin = 'AN', max_depth=8, n_estimators = 150, learning_rate = 0.07, subsample = 0.8, min_child_weight = 1, forecast2 = 'HWRF')


def compare_perf_intensity_per_year(dict, xgb_tot, xgb_total, year, forecast2, basin = 'AN', forecast = 'HWRF', mode = 'vmax'):
    if forecast2 != None:
        index = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline['cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        # X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_2 = baseline_[forecast2 + '_24_' + mode + '_7']
    else:
        index = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1].index#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        #X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    X_test_withBASELINE_total = X_test_total[index]
    baseline_1 = baseline_[forecast + '_24_'+mode+'_7']
    if mode == 'vmax':
        tgt_ = np.array(tgt_intensity_test[index] * std_ + mean_)
        print("Total number of steps for comparison: ", len(tgt_))
        preds_1 = xgb_tot.predict(X_test_withBASELINE_total) * std_ + mean_
        preds = xgb_total.predict(X_test_withBASELINE_total) * std_ + mean_
        #print("MAE intensity basin " + basin + " X stat vs "+ forecast + " : ", mean_absolute_error(tgt_intensity_test_withBASELINE * std_ + mean_,
                                                     #xgb.predict(X_test_withBASELINE) * std_ + mean_))
        print("Year ", year, " MAE intensity basin " + basin + " Hurricast trained full: ", np.around(mean_absolute_error(tgt_, preds), decimals = 2), "with std ", np.around(np.std(tgt_ - preds), decimals=2))
        print("Year ", year, " MAE intensity basin " + basin + " Hurricast trained until 2012: ", np.around(mean_absolute_error(tgt_, preds_1), decimals = 2), "with std ", np.around(np.std(tgt_ - preds_1), decimals=2))
        print("Year ", year, " MAE intensity basin " + basin + " Official Forecast "+ forecast + " : ",
              np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2), "with std ", np.around(np.std(tgt_ - baseline_1), decimals = 2))
        if forecast2 != None:
            print("Year ", year, " MAE intensity basin " + basin + " Official Forecast " + forecast2 + " : ",
                np.around(mean_absolute_error(tgt_, baseline_2), decimals=2), "with std ",
                np.around(np.std(tgt_ - baseline_2), decimals=2))
        append_dict_intensity(dict, tgt_, preds, preds_1, baseline_1, baseline_2, year)
        #print("Year ", year, " Percentage of missed intensification > 20kn Hurricast: ", np.around(sum(tgt_ - preds > 20)/len(preds) * 100, decimals = 2))
        #print("Year ", year, " Percentage of missed intensification > 20kn Official Forecast: ", np.around(sum(tgt_ - baseline_1 > 20) / len(baseline_1) * 100, decimals =2))


def compare_perf_track_per_year(dict, LATS_PRED_, LONS_PRED_, LATS_PRED_2012, LONS_PRED_2012, year, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    mode = 'lat'
    if forecast2 != None:
        index = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        LATS_BASE_2 = np.array(baseline_[forecast2 + '_24_lat_7'])
        LONS_BASE_2 = np.array(baseline_[forecast2 + '_24_lon_7'])
    else:
        index = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1].index#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    LATS_TEST_ = X_test['LAT_7'] + np.array(tgt_displacement_test[:, 0])*std_dx+mean_dx
    LONS_TEST_ = X_test['LON_7'] + np.array(tgt_displacement_test[:, 1])*std_dy+mean_dy
    baseline_1_x = baseline_[forecast + '_24_'+mode+'_7']
    baseline_1_y = baseline_[forecast + '_24_lon_7']
    LATS_BASE = np.array(baseline_1_x)
    LONS_BASE = np.array(baseline_1_y)
    LATS_TEST_ = np.array(LATS_TEST_[index])
    LONS_TEST_ = np.array(LONS_TEST_[index])
    LATS_PRED_ = np.array(LATS_PRED_[index])
    LONS_PRED_ = np.array(LONS_PRED_[index])
    LATS_PRED_2012 = np.array(LATS_PRED_2012[index])
    LONS_PRED_2012 = np.array(LONS_PRED_2012[index])
    d_km_baseline = np.zeros(len(LATS_BASE))
    d_km_baseline2 = np.zeros(len(LATS_BASE))
    d_km_pred = np.zeros(len(LONS_PRED_))
    d_km_pred_2012 = np.zeros(len(LONS_PRED_))
    for i in range(len(LATS_BASE)):
        d_km_baseline[i] = get_distance_km(LONS_BASE[i], LATS_BASE[i], LONS_TEST_[i], LATS_TEST_[i])
        d_km_pred[i] = get_distance_km(LONS_PRED_[i], LATS_PRED_[i], LONS_TEST_[i], LATS_TEST_[i])
        d_km_pred_2012[i] = get_distance_km(LONS_PRED_2012[i], LATS_PRED_2012[i], LONS_TEST_[i], LATS_TEST_[i])
        if forecast2 != None:
            d_km_baseline2[i] = get_distance_km(LONS_BASE_2[i], LATS_BASE_2[i], LONS_TEST_[i], LATS_TEST_[i])
    print("Year", year, "Number of timesteps:", len(LATS_BASE))
    print(basin, 'Model | MAE | std')
    print(forecast, np.around(d_km_baseline.mean(), decimals = 2), np.around(d_km_baseline.std(), decimals = 2))
    print(str(forecast2), np.around(d_km_baseline2.mean(), decimals = 2), np.around(d_km_baseline2.std(), decimals = 2))
    print("Hurricast Max Data", np.around(d_km_pred.mean(), decimals = 2), np.around(d_km_pred.std(), decimals = 2))
    print("Hurricast Until 2012", np.around(d_km_pred_2012.mean(), decimals=2), np.around(d_km_pred_2012.std(), decimals=2))
    print("\nModel | Number of Busts > 200km | Percentage Bust")
    print(forecast, sum(d_km_baseline > 200), np.around(sum(d_km_baseline > 200)*100/len(LATS_BASE), decimals = 2))
    print(str(forecast2), sum(d_km_baseline2 > 200), np.around(sum(d_km_baseline2 > 200)*100/len(LATS_BASE), decimals =2))
    print("Hurricast", sum(d_km_pred > 200), np.around(sum(d_km_pred > 200)*100/len(LATS_BASE), decimals = 2))
    append_dict_track(dict, d_km_baseline, d_km_baseline2, d_km_pred, d_km_pred_2012, year)

def append_dict_track(dict, d_km_baseline, d_km_baseline2, d_km_pred, d_km_pred_2012, year):
    dict['year'].append(year)
    dict['num_samples'].append(len(d_km_baseline))
    dict['MAEs_full'].append(np.around(d_km_pred.mean(), decimals = 2))
    dict['std_full'].append(np.around(d_km_pred.std(), decimals = 2))
    dict['MAES_2012'].append(np.around(d_km_pred_2012.mean(), decimals = 2))
    dict['std_2012'].append(np.around(d_km_pred_2012.std(), decimals=2))
    dict['MAES_SHIP'].append(np.around(d_km_baseline.mean(), decimals = 2))
    dict['std_SHIP'].append(np.around(d_km_baseline.std(), decimals = 2))
    dict['MAES_HWRF'].append(np.around(d_km_baseline2.mean(), decimals =2))
    dict['std_HWRF'].append(np.around(d_km_baseline2.std(), decimals=2))

def append_dict_intensity(dict, tgt_, preds, preds_1, baseline_1, baseline_2, year):
    dict['year'].append(year)
    dict['num_samples'].append(len(tgt_))
    dict['MAEs_full'].append(np.around(mean_absolute_error(tgt_, preds), decimals = 2))
    dict['std_full'].append(np.around(np.std(tgt_ - preds), decimals = 2))
    dict['MAES_2012'].append(np.around(mean_absolute_error(tgt_, preds_1), decimals = 2))
    dict['std_2012'].append(np.around(np.std(tgt_ - preds_1), decimals=2))
    dict['MAES_SHIP'].append(np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2))
    dict['std_SHIP'].append(np.around(np.std(tgt_ - baseline_1), decimals = 2))
    dict['MAES_HWRF'].append(np.around(mean_absolute_error(tgt_, baseline_2), decimals=2))
    dict['std_HWRF'].append(np.around(np.std(tgt_ - baseline_2), decimals=2))


def train_xgb_intensity_all_years(forecast2 = None, basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'HWRF'):
    train = X_train_total
    #test = X_test_total
    tgt_train = tgt_intensity_train
    if sparse:
        train = X_train_total_sparse_x
        #test = X_test_total_sparse_x
    if basin_only:
        train = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        tgt_train = tgt_intensity_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_total.fit(train, tgt_train)
    for year in range(2012, 2020):
        try:
            compare_perf_intensity_per_year(forecast2 = forecast2, xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', year = year)
            print("\n")
        except:
            print("\n No forecasts for year ", year)

def train_xgb_intensity_all_years_full_train(forecast2 = None, basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'HWRF'):
    train = X_train_total
    #test = X_test_total
    tgt_train = tgt_intensity_train
    if sparse:
        train = X_train_total_sparse_x
        #test = X_test_total_sparse_x
    if basin_only:
        train = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        tgt_train = tgt_intensity_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_total_1 = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
    xgb_total_1.fit(train, tgt_train)
    dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[]}
    for year in range(2012, 2020):
        try:
            index = X_test_baseline.loc[X_test_baseline['YEAR'] < year].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
            X_test_to_train = X_test_total[index]
            train = np.concatenate((X_train_total, X_test_to_train), axis = 0)
            tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[index]), axis = 0)
            xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                     subsample=subsample, min_child_weight=min_child_weight)
            xgb_total.fit(train, tgt_train)
            compare_perf_intensity_per_year(dict = dict, xgb_tot = xgb_total_1, forecast2 = forecast2, xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', year = year)
            print("\n")
        except:
            print("\n No forecasts for year ", year)
    return dict
